In [2]:
import splitfolders
import shutil

In [ ]:
path = "/kaggle/input/brain-tumor-classification-mri/Training"

# split data into 80% training and 20% validation
splitfolders.ratio(path, output="/kaggle/working/dataset", seed=42, ratio=(0.8, 0.2))

# the training portion called training we will leave as is and us for training
shutil.copytree("/kaggle/input/brain-tumor-classification-mri/Testing","/kaggle/working/dataset/test")
